https://lovit.github.io/nlp/graph/2017/08/21/ford_for_pos/ 의 예시코드 입니다.

In [1]:
sentence = '청하는 아이오아이의 출신입니다'

pos2words = {
    'Noun': set('아이 아이오 아이오아이 청하 출신 청'.split()),
    'Josa': set('은 는 이 가 의 를 을'.split()),
    'Verb': set('청하 이 있 하 했 입'.split()),
    'Eomi': set('다 었다 는 니다'.split())
}

## Dictionary

In [2]:
class Dictionary:
    def __init__(self, pos2words=None):
        self.pos2words = pos2words if pos2words else {}
        self.max_len = self._set_max_len()

    def _set_max_len(self):
        if not self.pos2words: return 0
        return max((len(word) for words in self.pos2words.values() for word in words))

    def get_pos(self, word):
        return [pos for pos, words in self.pos2words.items() if word in words]

dictionary = Dictionary(pos2words)

In [3]:
dictionary.get_pos('이')

['Josa', 'Verb']

In [4]:
dictionary.get_pos('청하')

['Noun', 'Verb']

## Word graph

### Lookup

In [5]:
def word_lookup(eojeol, offset):
    n = len(eojeol)
    words = [[] for _ in range(n)]
    for b in range(n):
        for r in range(1, dictionary.max_len+1):
            e = b+r
            if e > n:
                continue
            sub = eojeol[b:e]
            for pos in dictionary.get_pos(sub):
                words[b].append((sub, pos, b+offset, e+offset))
    return words

word_lookup('청하는', 0)

[[('청', 'Noun', 0, 1), ('청하', 'Noun', 0, 2), ('청하', 'Verb', 0, 2)],
 [('하', 'Verb', 1, 2)],
 [('는', 'Josa', 2, 3), ('는', 'Eomi', 2, 3)]]

In [6]:
word_lookup('청하는', 3)

[[('청', 'Noun', 3, 4), ('청하', 'Noun', 3, 5), ('청하', 'Verb', 3, 5)],
 [('하', 'Verb', 4, 5)],
 [('는', 'Josa', 5, 6), ('는', 'Eomi', 5, 6)]]

In [7]:
def lookup(sentence):
    sent = []
    for eojeol in sentence.split():
        sent += word_lookup(eojeol, offset=len(sent))
    return sent

lookup(sentence)

[[('청', 'Noun', 0, 1), ('청하', 'Noun', 0, 2), ('청하', 'Verb', 0, 2)],
 [('하', 'Verb', 1, 2)],
 [('는', 'Josa', 2, 3), ('는', 'Eomi', 2, 3)],
 [('아이', 'Noun', 3, 5), ('아이오', 'Noun', 3, 6), ('아이오아이', 'Noun', 3, 8)],
 [('이', 'Josa', 4, 5), ('이', 'Verb', 4, 5)],
 [],
 [('아이', 'Noun', 6, 8)],
 [('이', 'Josa', 7, 8), ('이', 'Verb', 7, 8)],
 [('의', 'Josa', 8, 9)],
 [('출신', 'Noun', 9, 11)],
 [],
 [('입', 'Verb', 11, 12)],
 [('니다', 'Eomi', 12, 14)],
 [('다', 'Eomi', 13, 14)]]

### Edges

In [8]:
def draw_edges(sentence):
    chars = sentence.replace(' ', '')
    sent = lookup(sentence)
    n_char = len(chars)
    sent.append([('EOS', 'EOS', n_char + 1, n_char + 1)])

    nonempty_first = get_nonempty_first(sent, offset=0)
    if nonempty_first > 0:
        sent[0].append((chars[:nonempty_first], 'Unk', 0, nonempty_first))

    edges = forward_link(sent, chars)
    edges = unk_link(edges, sent)
    edges = add_bos(edges, sent)

    edges = sorted(edges, key=lambda x:(x[0][2], x[0][3], x[1][2]))
    return edges, sent

def forward_link(sent, chars):
    edges = []
    for words in sent[:-1]:
        for word in words:
            begin = word[2]
            end = word[3]
            if not sent[end]:
                next_begin = get_nonempty_first(sent, end)
                unk = (chars[end:next_begin], 'Unk', end, next_begin)
                edges.append((word, unk))
            else:
                for adjacent in sent[end]:
                    edges.append((word, adjacent))
    return edges

def unk_link(edges, sent):
    unks = {node for _, node in edges if node[1] == 'Unk'}
    for unk in unks:
        for adjacent in sent[unk[3]]:
            edges.append((unk, adjacent))
    return edges

def add_bos(edges, sent):
    bos = ('BOS', 'BOS', 0, 0)
    for word in sent[0]:
        edges.append((bos, word))
    return edges

def get_nonempty_first(sent, offset=0):
    for i in range(offset, len(sent)+1):
        if sent[i]:
            return i
    return offset


In [9]:
edges, sent = draw_edges(sentence)

In [10]:
sent

[[('청', 'Noun', 0, 1), ('청하', 'Noun', 0, 2), ('청하', 'Verb', 0, 2)],
 [('하', 'Verb', 1, 2)],
 [('는', 'Josa', 2, 3), ('는', 'Eomi', 2, 3)],
 [('아이', 'Noun', 3, 5), ('아이오', 'Noun', 3, 6), ('아이오아이', 'Noun', 3, 8)],
 [('이', 'Josa', 4, 5), ('이', 'Verb', 4, 5)],
 [],
 [('아이', 'Noun', 6, 8)],
 [('이', 'Josa', 7, 8), ('이', 'Verb', 7, 8)],
 [('의', 'Josa', 8, 9)],
 [('출신', 'Noun', 9, 11)],
 [],
 [('입', 'Verb', 11, 12)],
 [('니다', 'Eomi', 12, 14)],
 [('다', 'Eomi', 13, 14)],
 [('EOS', 'EOS', 15, 15)]]

In [11]:
edges

[(('BOS', 'BOS', 0, 0), ('청', 'Noun', 0, 1)),
 (('BOS', 'BOS', 0, 0), ('청하', 'Noun', 0, 2)),
 (('BOS', 'BOS', 0, 0), ('청하', 'Verb', 0, 2)),
 (('청', 'Noun', 0, 1), ('하', 'Verb', 1, 2)),
 (('청하', 'Noun', 0, 2), ('는', 'Josa', 2, 3)),
 (('청하', 'Noun', 0, 2), ('는', 'Eomi', 2, 3)),
 (('청하', 'Verb', 0, 2), ('는', 'Josa', 2, 3)),
 (('청하', 'Verb', 0, 2), ('는', 'Eomi', 2, 3)),
 (('하', 'Verb', 1, 2), ('는', 'Josa', 2, 3)),
 (('하', 'Verb', 1, 2), ('는', 'Eomi', 2, 3)),
 (('는', 'Josa', 2, 3), ('아이', 'Noun', 3, 5)),
 (('는', 'Josa', 2, 3), ('아이오', 'Noun', 3, 6)),
 (('는', 'Josa', 2, 3), ('아이오아이', 'Noun', 3, 8)),
 (('는', 'Eomi', 2, 3), ('아이', 'Noun', 3, 5)),
 (('는', 'Eomi', 2, 3), ('아이오', 'Noun', 3, 6)),
 (('는', 'Eomi', 2, 3), ('아이오아이', 'Noun', 3, 8)),
 (('아이', 'Noun', 3, 5), ('오', 'Unk', 5, 6)),
 (('아이오', 'Noun', 3, 6), ('아이', 'Noun', 6, 8)),
 (('아이오아이', 'Noun', 3, 8), ('의', 'Josa', 8, 9)),
 (('이', 'Josa', 4, 5), ('오', 'Unk', 5, 6)),
 (('이', 'Verb', 4, 5), ('오', 'Unk', 5, 6)),
 (('오', 'Unk', 5, 6), ('아이'

### Weight

In [12]:
transition = {
    ('Noun', 'Josa'): 0.7,
    ('Noun', 'Noun'): 0.3,
    ('Verb', 'Eomi'): 0.5,
    ('Verb', 'Noun'): 0.5,
    ('Verb', 'Josa'): -0.1,
}
generation = {
    'Noun': {
        '아이오아이': 0.5,
        '청하': 0.2,
    }
}

In [13]:
class Weighter:
    def __init__(self, transition, generation):
        self.transition = transition
        self.generation = generation

    def cost(self, edge):
        score = 0
        score += self.transition.get((edge[0][1], edge[1][1]), 0)
        score += self.generation.get(edge[1][1], {}).get(edge[1][0], 0)
        return -1 * score

weighter = Weighter(transition, generation)

In [14]:
print(edges[4])
print(weighter.cost(edges[4]))

(('청하', 'Noun', 0, 2), ('는', 'Josa', 2, 3))
-0.7


In [15]:
def attach_weight(edges):
    edges = [(edge[0], edge[1], weighter.cost(edge)) for edge in edges]
    return edges

edges = attach_weight(edges)
edges

[(('BOS', 'BOS', 0, 0), ('청', 'Noun', 0, 1), 0),
 (('BOS', 'BOS', 0, 0), ('청하', 'Noun', 0, 2), -0.2),
 (('BOS', 'BOS', 0, 0), ('청하', 'Verb', 0, 2), 0),
 (('청', 'Noun', 0, 1), ('하', 'Verb', 1, 2), 0),
 (('청하', 'Noun', 0, 2), ('는', 'Josa', 2, 3), -0.7),
 (('청하', 'Noun', 0, 2), ('는', 'Eomi', 2, 3), 0),
 (('청하', 'Verb', 0, 2), ('는', 'Josa', 2, 3), 0.1),
 (('청하', 'Verb', 0, 2), ('는', 'Eomi', 2, 3), -0.5),
 (('하', 'Verb', 1, 2), ('는', 'Josa', 2, 3), 0.1),
 (('하', 'Verb', 1, 2), ('는', 'Eomi', 2, 3), -0.5),
 (('는', 'Josa', 2, 3), ('아이', 'Noun', 3, 5), 0),
 (('는', 'Josa', 2, 3), ('아이오', 'Noun', 3, 6), 0),
 (('는', 'Josa', 2, 3), ('아이오아이', 'Noun', 3, 8), -0.5),
 (('는', 'Eomi', 2, 3), ('아이', 'Noun', 3, 5), 0),
 (('는', 'Eomi', 2, 3), ('아이오', 'Noun', 3, 6), 0),
 (('는', 'Eomi', 2, 3), ('아이오아이', 'Noun', 3, 8), -0.5),
 (('아이', 'Noun', 3, 5), ('오', 'Unk', 5, 6), 0),
 (('아이오', 'Noun', 3, 6), ('아이', 'Noun', 6, 8), -0.3),
 (('아이오아이', 'Noun', 3, 8), ('의', 'Josa', 8, 9), -0.7),
 (('이', 'Josa', 4, 5), ('오', '

In [16]:
from collections import defaultdict
from pprint import pprint

def edges_to_dict(edges):
    g = defaultdict(lambda: {})
    for from_, to_, weight in edges:
        g[from_][to_] = weight
    return dict(g)

g = edges_to_dict(edges)
pprint(g)

{('BOS', 'BOS', 0, 0): {('청', 'Noun', 0, 1): 0,
                        ('청하', 'Noun', 0, 2): -0.2,
                        ('청하', 'Verb', 0, 2): 0},
 ('는', 'Eomi', 2, 3): {('아이', 'Noun', 3, 5): 0,
                       ('아이오', 'Noun', 3, 6): 0,
                       ('아이오아이', 'Noun', 3, 8): -0.5},
 ('는', 'Josa', 2, 3): {('아이', 'Noun', 3, 5): 0,
                       ('아이오', 'Noun', 3, 6): 0,
                       ('아이오아이', 'Noun', 3, 8): -0.5},
 ('니다', 'Eomi', 12, 14): {('EOS', 'EOS', 15, 15): 0},
 ('다', 'Eomi', 13, 14): {('EOS', 'EOS', 15, 15): 0},
 ('아이', 'Noun', 3, 5): {('오', 'Unk', 5, 6): 0},
 ('아이', 'Noun', 6, 8): {('의', 'Josa', 8, 9): -0.7},
 ('아이오', 'Noun', 3, 6): {('아이', 'Noun', 6, 8): -0.3},
 ('아이오아이', 'Noun', 3, 8): {('의', 'Josa', 8, 9): -0.7},
 ('오', 'Unk', 5, 6): {('아이', 'Noun', 6, 8): 0},
 ('의', 'Josa', 8, 9): {('출신', 'Noun', 9, 11): 0},
 ('이', 'Josa', 4, 5): {('오', 'Unk', 5, 6): 0},
 ('이', 'Josa', 7, 8): {('의', 'Josa', 8, 9): 0},
 ('이', 'Verb', 4, 5): {('오', 'Unk', 5

In [17]:
import sys
sys.path.append('../')
from shortestpath import ford

In [18]:
bos = ('BOS', 'BOS', 0, 0)
eos = ('EOS', 'EOS', 15, 15)
ford(g, bos, eos, debug=True)

cost[('BOS', 'BOS', 0, 0) -> ('청', 'Noun', 0, 1)] = 24.200000000000003 -> 0
cost[('BOS', 'BOS', 0, 0) -> ('청하', 'Noun', 0, 2)] = 24.200000000000003 -> -0.2
cost[('BOS', 'BOS', 0, 0) -> ('청하', 'Verb', 0, 2)] = 24.200000000000003 -> 0
cost[('청', 'Noun', 0, 1) -> ('하', 'Verb', 1, 2)] = 24.200000000000003 -> 0
cost[('청하', 'Noun', 0, 2) -> ('는', 'Josa', 2, 3)] = 24.200000000000003 -> -0.8999999999999999
cost[('청하', 'Noun', 0, 2) -> ('는', 'Eomi', 2, 3)] = 24.200000000000003 -> -0.2
cost[('청하', 'Verb', 0, 2) -> ('는', 'Eomi', 2, 3)] = -0.2 -> -0.5
cost[('는', 'Josa', 2, 3) -> ('아이', 'Noun', 3, 5)] = 24.200000000000003 -> -0.8999999999999999
cost[('는', 'Josa', 2, 3) -> ('아이오', 'Noun', 3, 6)] = 24.200000000000003 -> -0.8999999999999999
cost[('는', 'Josa', 2, 3) -> ('아이오아이', 'Noun', 3, 8)] = 24.200000000000003 -> -1.4
cost[('아이', 'Noun', 3, 5) -> ('오', 'Unk', 5, 6)] = 24.200000000000003 -> -0.8999999999999999
cost[('아이오', 'Noun', 3, 6) -> ('아이', 'Noun', 6, 8)] = 24.200000000000003 -> -1.2
cost[('아이

{'cost': -2.5999999999999996,
 'paths': [[('BOS', 'BOS', 0, 0),
   ('청하', 'Noun', 0, 2),
   ('는', 'Josa', 2, 3),
   ('아이오아이', 'Noun', 3, 8),
   ('의', 'Josa', 8, 9),
   ('출신', 'Noun', 9, 11),
   ('입', 'Verb', 11, 12),
   ('니다', 'Eomi', 12, 14),
   ('EOS', 'EOS', 15, 15)]]}